In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='********-****-******-****-**********', project_access_token='p-************************************')
pc = project.project_context


In [2]:
%%capture
!pip install yfinance==0.1.54
!pip install pandas-datareader==0.8.1

In [3]:
from pandas_datareader import data as pdr
from datetime import date,timedelta
import yfinance as yf
yf.pdr_override()
import pandas as pd
import datetime 

today = date.today()
today = today + timedelta(days=1)

In [5]:
date_time = '2005-04-05'
start = datetime.datetime.strptime(date_time, '%Y-%d-%m') 

In [7]:
### Get the datas (Itau share price, ibovespa, USDBRL)
TS_itau = pdr.get_data_yahoo('itub4.sa', start=start, end=today).reset_index()
BVSP = pdr.get_data_yahoo('^BVSP', start=start, end=today).reset_index()
USDBRL = pdr.get_data_yahoo('USDBRL=X', start=start, end=today).reset_index()

### Cleanning and group by
TS_itau= TS_itau[['Date','Open','Close']]
BVSP= BVSP[['Date','Open']]
USDBRL= USDBRL[['Date','Open']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
TS_itau.columns = ['Date','Itau_open','Itau_Close']
BVSP.columns = ['Date','BVSP_open']
USDBRL.columns = ['Date','USDBRL_open']
result = pd.merge(TS_itau, BVSP, on='Date')
result = pd.merge(result, USDBRL, on='Date')

In [9]:
TS_itau.tail()

,Date,Itau_open,Itau_Close
3752,2020-07-21,27.600000,27.780001
3753,2020-07-22,27.850000,27.070000
3754,2020-07-23,27.100000,26.500000
3755,2020-07-24,26.350000,26.510000
3756,2020-07-27,26.629999,27.610001


In [10]:
s = result['Itau_Close']
lag_col = pd.concat([s.shift(), s.shift(2),s.shift(3)], axis=1)
lag_col.columns = ['lag_1','lag_2','lag_3']
result = pd.concat([result, lag_col], axis=1, sort=False).dropna()

In [11]:
result.head()

,Date,Itau_open,Itau_Close,BVSP_open,USDBRL_open,lag_1,lag_2,lag_3
3,2006-03-29,10.9282,10.8934,36684.0,2.2270,10.9282,11.1600,11.4829
4,2006-03-30,11.0093,10.8405,37493.0,2.2117,10.8934,10.9282,11.1600
5,2006-03-31,10.8620,10.5971,37783.0,2.1853,10.8405,10.8934,10.9282
6,2006-04-03,10.7047,11.0358,37952.0,2.1618,10.5971,10.8405,10.8934
7,2006-04-04,11.0789,11.2263,38718.0,2.1370,11.0358,10.5971,10.8405


In [1]:
project.save_data(data=result.to_csv(), file_name='ts_data.csv',overwrite=True)

NameError: name 'project' is not defined